In [1]:
# Dependencies
from bs4 import BeautifulSoup as bs
from splinter import Browser
import pandas as pd
import time
from selenium import webdriver

In [2]:
mars = {}

In [3]:
# Chromedriver
!which chromedriver

which: no chromedriver in (/c/Users/lydiapf/anaconda3/envs/PythonData:/c/Users/lydiapf/anaconda3/envs/PythonData/Library/mingw-w64/bin:/c/Users/lydiapf/anaconda3/envs/PythonData/Library/usr/bin:/c/Users/lydiapf/anaconda3/envs/PythonData/Library/bin:/c/Users/lydiapf/anaconda3/envs/PythonData/Scripts:/c/Users/lydiapf/anaconda3/envs/PythonData/bin:C:/Users/lydiapf/anaconda3:C:/Users/lydiapf/anaconda3/Library/mingw-w64/bin:C:/Users/lydiapf/anaconda3/Library/usr/bin:C:/Users/lydiapf/anaconda3/Library/bin:C:/Users/lydiapf/anaconda3/Scripts:C:/Users/lydiapf/anaconda3/bin:C:/Users/lydiapf/anaconda3/Scripts/condabin:/c/Users/lydiapf/bin:/mingw64/bin:/usr/local/bin:/usr/bin:/usr/bin:/mingw64/bin:/usr/bin:/c/Users/lydiapf/bin:/c/Windows/system32:/c/Windows:/c/Windows/System32/Wbem:/c/Windows/System32/WindowsPowerShell/v1.0:/c/Windows/System32/OpenSSH:/c/Program Files/Condusiv Technologies/ExpressCache:/c/Program Files (x86)/PharosSystems/Core:/c/Program Files/Intel/WiFi/bin:/c/Program Files/Commo

In [4]:
# NASA Mars News

In [5]:
executable_path = {'executable_path': 'chromedriver'}
browser = Browser('chrome', **executable_path, headless=False)
browser.visit('https://mars.nasa.gov/news/')

In [6]:
# Parse the html result with soup 
html = browser.html
soup = bs(html, 'html.parser')

In [7]:
# 
news_title = soup.select_one('div.content_title a').text

In [8]:
# Retrieve the latest paragraph
news_p = soup.select_one('div.article_teaser_body').text
mars["news_title"] = news_title
mars["news_p"] = news_p

In [9]:
# Create print statement of the latest news title and paragraph text
print(f'The latest news title is: {news_title} \nThe paragraph text is: {news_p}')

The latest news title is: MOXIE Could Help Future Rockets Launch Off Mars 
The paragraph text is: NASA's Perseverance rover carries a device to convert Martian air into oxygen that, if produced on a larger scale, could be used not just for breathing, but also for fuel.


In [10]:
# JPL Mars Space Images - Featured Image

In [11]:
#Visit Image Url
img_url = 'https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars'
browser.visit(img_url)

In [12]:
# Find the full image botton and click
img_elem = browser.find_by_id('full_image')
img_elem.click()

In [13]:
# Find the more info button and click
browser.is_element_present_by_text('more info', wait_time=3)
more_info_elem = browser.links.find_by_partial_text('more info')
more_info_elem.click()

In [14]:
# Parse the html result with soup
img_html = browser.html
img_soup = bs(img_html, 'html.parser')

In [15]:
# Find the relative image url
img_url_rel = img_soup.select_one('figure.lede a img').get("src")
img_url_rel

'/spaceimages/images/largesize/PIA18848_hires.jpg'

In [16]:
# Use the base url to create an absolute url
img_url = f'https://www.jpl.nasa.gov{img_url_rel}'
img_url
mars["featured_image"] = img_url

In [17]:
# Mars Facts

In [18]:
# URL
facts_url = 'https://space-facts.com/mars/'

# Read html by Pandas
tables = pd.read_html(facts_url)

# Slice off dataframes that needed
mars_df = tables[0]

# Set columns
mars_df.columns = ['Description', 'Mars']
mars_df.set_index('Description', inplace=True)

mars_html = mars_df.to_html()

mars_html

mars["facts"] = mars_html

In [19]:
hem_list = ['Cerberus', 'Schiaparelli', 'Syrtis', 'Valles']
hemisphere_image_urls = []

for hem in hem_list:
    # Visit URL
    hem_url = 'https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars'
    browser.visit(hem_url)
    
    # Find the hemisphere images and click
    hem_elem = browser.links.find_by_partial_text(hem)
    hem_elem.click()
    
    # Find original to download full img
    hem_elem_org = browser.find_by_text("Original", wait_time=1)
    hem_elem_org.click()
    
    # Scrape the Mars website 
    hem_html = browser.html
    hem_soup = bs(hem_html, 'html.parser')
    
    hem_title = hem_soup.h2.text
    hem_url = hem_soup.select_one('li a')['href']
    
    hem_dict = {
        'title': hem_title,
        'img_url': hem_url
    }
    
    hemisphere_image_urls.append(hem_dict)

In [20]:
hemisphere_image_urls

mars["hemispheres"] = hemisphere_image_urls